# Julia - Day 5

https://adventofcode.com/2021/day/5

In [101]:
using DataFrames

## Data

In [102]:
function import_data(input_path)
    input = readlines(input_path)
    input = split.(input, " -> ")

    coord1 = [row[1] for row in input]
    coord1 = split.(coord1, ",")
    x1 = [parse(Int, row[1]) for row in coord1]
    y1 = [parse(Int, row[2]) for row in coord1]

    coord2 = [row[2] for row in input]
    coord2 = split.(coord2, ",")
    x2 = [parse(Int, row[1]) for row in coord2]
    y2 = [parse(Int, row[2]) for row in coord2]

    return DataFrame(
        x1 = x1,
        y1 = y1,
        x2 = x2,
        y2 = y2        
    )
end

import_data (generic function with 1 method)

In [103]:
example = import_data("Input/5_example.txt")
input = import_data("Input/5_1.txt")


,x1,y1,x2,y2
,Int64,Int64,Int64,Int64
1,989,854,521,854
2,831,695,402,266
3,38,805,306,537
4,802,24,802,824
5,951,478,951,758
6,508,987,508,868
7,602,246,108,246
8,765,781,76,92
9,248,757,644,361


In [152]:
example

,x1,y1,x2,y2
,Int64,Int64,Int64,Int64
1,0,9,5,9
2,8,0,0,8
3,9,4,3,4
4,2,2,2,1
5,7,0,7,4
6,6,4,2,0
7,0,9,2,9
8,3,4,1,4
9,0,0,8,8


In [153]:
ocean_floor = create_ocean_floor(example)
ocean_floor

10×10 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0

In [154]:
coords = coord_between_points(0, 9, 5, 9)

6×2 Matrix{Int64}:
 0  9
 1  9
 2  9
 3  9
 4  9
 5  9

In [155]:
update_ocean_floor!(ocean_floor, coords)

In [156]:
ocean_floor

10×10 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  0  0  0  0

## Part 1

### Function

In [163]:
function vertical_or_horizontal(data)
    return (data[:, :x1] .== data[:, :x2]) .| (data[:, :y1] .== data[:, :y2])
end

function create_ocean_floor(data)
    x_max = maximum(vcat(data[:, :x1], data[:, :x2], 1)) + 1 #because coord start at 0 and matrix start at 1
    y_max = maximum(vcat(data[:, :y1], data[:, :y2], 1)) + 1
    return zeros(Int, x_max, y_max)
end

function line_function_creator(x1, y1, x2, y2)
    #when not a vertical line where b = inf
    if x1 != x2
        b = (y2 - y1) / (x2 - x1)
        a = y1 - b * x1
        return line_function(x)::Int64 = a + b*x
    end

end

function coord_between_points(x1, y1, x2, y2)
    
    coords = zeros(Int, 0, 2)

    if (x1 == x2) & (y1 == y2)
        coords = vcat(coords, [])

    elseif x1 == x2
        for y in range(start = y1, stop = y2, step = y2 - y1 > 0 ? 1 : -1)
            coords = vcat(coords, [x1 y])
        end
    
    elseif y1 == y2
        for x in range(start = x1, stop = x2, step = x2 - x1 > 0 ? 1 : -1)
            coords = vcat(coords, [x y1])
        end

    else
        line = line_function_creator(x1, y1, x2, y2)
        for x in range(start = x1, stop = x2, step = x2 - x1 > 0 ? 1 : -1)
            coords = vcat(coords, [x line(x)])
        end

    end

    return coords
end

function update_ocean_floor!(ocean_floor, coords)
    for coord_pair in eachrow(coords)
        ocean_floor[coord_pair[2] + 1, coord_pair[1] + 1] += 1
        #x is col and y is row
    end
end

function ocean_floor_survey(data)

    data = data[vertical_or_horizontal(data), :]
    ocean_floor = create_ocean_floor(data)

    for row in eachrow(data)

        coords = coord_between_points(row[:x1], row[:y1], row[:x2], row[:y2])
        update_ocean_floor!(ocean_floor, coords)

    end

    return ocean_floor

end

function n_dangerous_area(ocean_floor, threshold = 2)
    return length(findall(ocean_floor .>= threshold))
end

n_dangerous_area (generic function with 2 methods)

### Result

In [166]:
#Example
result = ocean_floor_survey(example)
result = n_dangerous_area(result)

println(
    string("Example : ", result)
)
#Input
result = ocean_floor_survey(input)
result = n_dangerous_area(result)

println(
    string("Input : ", result)
)

Example : 5
Input : 5169


## Part 2

### Function

In [167]:
function ocean_floor_survey(data)

    ocean_floor = create_ocean_floor(data)

    for row in eachrow(data)

        coords = coord_between_points(row[:x1], row[:y1], row[:x2], row[:y2])
        update_ocean_floor!(ocean_floor, coords)

    end

    return ocean_floor

end

ocean_floor_survey (generic function with 1 method)

### Result

In [169]:
#Example
result = ocean_floor_survey(example)
result = n_dangerous_area(result)

println(
    string("Example : ", result)
)
#Input
result = ocean_floor_survey(input)
result = n_dangerous_area(result)

println(
    string("Input : ", result)
)

Example : 12
Input : 22083
